Import SAS File

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import saspy

import scipy.stats as sts

import statsmodels.formula.api as smf
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
os.getcwd()

'C:\\Users\\lavada.blanton\\Desktop\\Exercise'

In [3]:
data = pd.read_excel('BFP_Data_2021.xlsx')

Print Top 5 Rows

In [4]:
data.head()

,id,colrcode,reliab,time,av_br,av_spec,price,credit,av_pay,return,...,fn_newpr,fn_need,fn_order,fn_pric,fn_other,pr_area,sales,num_emp,industry,size
0,1,8.0,8.0,8.0,6.0,7.0,7.0,5.0,8.0,7.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5.0,6.0,34.0,large
1,5,8.0,9.0,9.0,9.0,9.0,9.0,7.0,6.0,8.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,5.0,30.0,NaN
2,9,5.0,8.0,7.0,4.0,8.0,5.0,1.0,1.0,5.0,...,0.0,1.0,0.0,0.0,0.0,2.0,NaN,3.0,34.0,NaN
3,10,8.0,9.0,9.0,6.0,7.0,9.0,9.0,4.0,9.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,4.0,99.0,NaN
4,11,8.0,9.0,8.0,7.0,8.0,4.0,3.0,2.0,4.0,...,0.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,35.0,large


Print ALL Variable Names

In [5]:
data.columns

Index(['id', 'colrcode', 'reliab', 'time', 'av_br', 'av_spec', 'price',
       'credit', 'av_pay', 'return', 'warranty', 'talk_dir', 'reliab2',
       'time2', 'av_br2', 'av_spec2', 'price2', 'credit2', 'return2',
       'warrant2', 'satisf', 'rate', 'percent', 'i_acces', 'purch_i',
       'purch_i2', 'purchgrp', 'often', 'rout_int', 'rout_cal', 'rout_sal',
       'rout_no', 'fn_tech', 'fn_newpr', 'fn_need', 'fn_order', 'fn_pric',
       'fn_other', 'pr_area', 'sales', 'num_emp', 'industry', 'size'],
      dtype='object')

Print Summary Statistics for RATE

In [7]:
data['rate'].describe().T

count    308.000000
mean       6.795455
std        1.640404
min        1.000000
25%        6.000000
50%        7.000000
75%        8.000000
max        9.000000
Name: rate, dtype: float64

Stepwise Regression with RATE as DV and Reliab2-Warrant2 as IV

In [10]:
model_data = data[['rate','reliab2','time2','av_br2','av_spec2','price2','credit2', 'return2', 'warrant2']]

In [11]:
model_data.isnull().sum()

rate         9
reliab2     11
time2       11
av_br2      13
av_spec2    15
price2      13
credit2     23
return2     20
warrant2    18
dtype: int64

In [13]:
##This calls on a random forest imputation
import sys
import sklearn.neighbors._base 
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

imputer = MissForest()

In [14]:
model_imputed = imputer.fit_transform(model_data)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [17]:
model_data['rate_imp'] = model_imputed[:,0]
model_data['reliab2_imp'] = model_imputed[:,1]
model_data['time2_imp'] = model_imputed[:,2]
model_data['av_br2_imp'] = model_imputed[:,3]
model_data['av_spec2_imp'] = model_imputed[:,4]
model_data['price2_imp'] = model_imputed[:,5]
model_data['credit2_imp'] = model_imputed[:,6]
model_data['return2_imp'] = model_imputed[:,7]
model_data['warrant2_imp'] = model_imputed[:,8]

<ipython-input-17-4da9d6d3d427>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['rate_imp'] = model_imputed[:,0]
<ipython-input-17-4da9d6d3d427>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data['reliab2_imp'] = model_imputed[:,1]
<ipython-input-17-4da9d6d3d427>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [18]:
model_data.head()

,rate,reliab2,time2,av_br2,av_spec2,price2,credit2,return2,warrant2,rate_imp,reliab2_imp,time2_imp,av_br2_imp,av_spec2_imp,price2_imp,credit2_imp,return2_imp,warrant2_imp
0,7.0,7.0,6.0,6.0,7.0,8.0,5.0,6.0,5.0,7.000000,7.0,6.0,6.0,7.0,8.0,5.0,6.0,5.0
1,6.0,5.0,4.0,8.0,8.0,7.0,5.0,6.0,7.0,6.000000,5.0,4.0,8.0,8.0,7.0,5.0,6.0,7.0
2,NaN,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.719239,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
3,2.0,3.0,3.0,6.0,6.0,9.0,6.0,6.0,6.0,2.000000,3.0,3.0,6.0,6.0,9.0,6.0,6.0,6.0
4,7.0,7.0,6.0,8.0,8.0,6.0,6.0,6.0,6.0,7.000000,7.0,6.0,8.0,8.0,6.0,6.0,6.0,6.0


In [19]:
##Check that all imputed variables are no longer missing values
model_data.isnull().sum()

rate             9
reliab2         11
time2           11
av_br2          13
av_spec2        15
price2          13
credit2         23
return2         20
warrant2        18
rate_imp         0
reliab2_imp      0
time2_imp        0
av_br2_imp       0
av_spec2_imp     0
price2_imp       0
credit2_imp      0
return2_imp      0
warrant2_imp     0
dtype: int64

In [20]:
model_data2 = model_data.filter(regex='imp')

In [21]:
model_data2.head()

,rate_imp,reliab2_imp,time2_imp,av_br2_imp,av_spec2_imp,price2_imp,credit2_imp,return2_imp,warrant2_imp
0,7.000000,7.0,6.0,6.0,7.0,8.0,5.0,6.0,5.0
1,6.000000,5.0,4.0,8.0,8.0,7.0,5.0,6.0,7.0
2,5.719239,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
3,2.000000,3.0,3.0,6.0,6.0,9.0,6.0,6.0,6.0
4,7.000000,7.0,6.0,8.0,8.0,6.0,6.0,6.0,6.0


In [22]:
model_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rate_imp      317 non-null    float64
 1   reliab2_imp   317 non-null    float64
 2   time2_imp     317 non-null    float64
 3   av_br2_imp    317 non-null    float64
 4   av_spec2_imp  317 non-null    float64
 5   price2_imp    317 non-null    float64
 6   credit2_imp   317 non-null    float64
 7   return2_imp   317 non-null    float64
 8   warrant2_imp  317 non-null    float64
dtypes: float64(9)
memory usage: 22.4 KB


In [30]:
Y= model_data2[['rate_imp']]
X= model_data2.drop(['rate_imp'], axis=1)

In [31]:
#stepwise regression function creation
def stepwise_selection(data, target, SL_in=0.05, SL_out=0.05):
    initial_features = data.columns.tolist()
    best_features=[]
    while (len(initial_features)>0):
        remaining_features= list(set(initial_features)-set(best_features))
        new_pval=pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model=sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<SL_in):
            best_features.append(new_pval.idxmin())
            while(len(best_features)>0):
                best_features_with_constant = sm.add_constant(data[best_features])
                p_values = sm.OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value=p_values.max()
                if(max_p_value>=SL_out):
                    excluded_feature = p_values.idxmax()
                    best_features.remove(excluded_feature)
                else:
                    break
        else:
            break
    return best_features

In [32]:
#this will print the important variables from the stepwise. 
stepwise_selection(X,Y)

<ipython-input-31-7de1ce197138>:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval=pd.Series(index=remaining_features)
<ipython-input-31-7de1ce197138>:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval=pd.Series(index=remaining_features)
<ipython-input-31-7de1ce197138>:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval=pd.Series(index=remaining_features)


['reliab2_imp', 'time2_imp']